<a href="https://colab.research.google.com/github/HosseinEyvazi/Supervised-Learning/blob/main/Titanic_w4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

titanic_path = kagglehub.competition_download('titanic')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Imports

In [ ]:
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler , OneHotEncoder , LabelEncoder , power_transform , QuantileTransformer
from sklearn.model_selection import train_test_split , RandomizedSearchCV , GridSearchCV
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix , classification_report
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')




###  Data load and Understanding


In [ ]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
train_data.head()


In [ ]:
train_data.shape


In [ ]:
train_data.isna().sum(axis=0)

In [ ]:
correlation = train_data.select_dtypes(include=['number']).corr()
sns.heatmap(correlation , annot=True , cmap='coolwarm')

In [ ]:
train_data.describe()

### EDA

In [ ]:
train_data.drop('PassengerId' , axis=1 , inplace =True)
test_data.drop('PassengerId' , axis=1 , inplace =True)

In [ ]:
train_data.duplicated().sum()

In [ ]:
# Check null values in train data
train_data.isna().sum(axis=0)

In [ ]:
# Check null values in test data
test_data.isna().sum()

### Preprocessing and feature engineering

In [ ]:
train_data.columns


null value handling

In [ ]:
print(train_data.isna().sum())
print('-----------------------------')
print(test_data.isna().sum())

important note : df.groupby returns a series

In [ ]:
train_data.groupby([ 'Pclass' , 'Sex' ])['Fare'].mean().index

In [ ]:
# Age null handling
train_data.Age = train_data.groupby([ 'Pclass' , 'Sex' ])['Age'].transform(lambda x:x.fillna(x.median()))
# train_data.groupby([ 'Pclass' , 'Sex' ])['Age'].transform(lambda x:x.fillna(x.mean())) # another way
test_data.Age=test_data.groupby([ 'Pclass' , 'Sex' ])['Age'].transform(lambda x:x.fillna(x.median()))

In [ ]:
#Cabin null handling
train_data.Cabin = train_data.Cabin.fillna('has not' )
train_data.Cabin[train_data['Cabin'] != 'has not'] = 'has'

test_data.Cabin = test_data.Cabin.fillna('has not' )
test_data.Cabin[test_data['Cabin'] != 'has not'] = 'has'

test_data.rename(columns={'Cabin':'Has_Cabin'})
train_data.rename(columns={'Cabin':'Has_Cabin'})

train_data.drop('Cabin' , axis=1)
test_data.drop('Cabin' , axis=1)


In [ ]:

train_data.Embarked.fillna(train_data.Embarked.mode()[0] , inplace=True) #Embarked is categorical column
test_data.Embarked.fillna(train_data.Embarked.mode()[0] , inplace=True) #Embarked is categorical column
train_data.Embarked.unique()

In [ ]:
test_data.Fare.fillna(test_data.Fare.mode()[0] , inplace=True)

In [ ]:
print(train_data.isna().sum().sum())
test_data.isna().sum().sum()

### Feature engineering

feature engineering for family size

In [ ]:
train_data['Family_Size'] = train_data['Parch']+ train_data['SibSp']
test_data['Family_Size'] = test_data['Parch']+ test_data['SibSp']

train_data['Is_Alone'] = train_data['Family_Size'] == 0
test_data['Is_Alone'] = test_data['Family_Size'] == 0

train_data.sample(5 , random_state=42)

ticket prefix extration and encoding

In [ ]:
def prefix_extraction(string):
    string = string.replace('/' , '').replace('.' , '')
    # print(string)
    parts=string.split()
    # print(string)
    return parts[0] if len(parts)>1 else None


# How to apply a function on a column
# new features creation
train_data['Ticket_length'] = train_data.Ticket.apply(lambda x: len(x))
test_data['Ticket_length'] = test_data.Ticket.apply(lambda x: len(x))

train_data['Ticket_Prefix']=train_data.Ticket.apply(prefix_extraction)
test_data['Ticket_Prefix']=test_data.Ticket.apply(prefix_extraction)
# train_data=train_data.rename(columns={'Ticket' : 'Ticket_Prefix'}  )

train_data

bining features

In [ ]:
train_data['Fare_binning'] = pd.qcut(
    train_data.Fare,
    q=[0, 0.33, 0.66, 1],  # Quantiles
    labels=['low', 'moderate', 'high']
)
# alternative implementation :
# train_data['Fare_binning'] = pd.cut(
#     train_data.Fare,
#     bins=[0, train_data.Fare.quantile(0.33), train_data.Fare.quantile(0.66), train_data.Fare.max()],
#     labels=['low', 'moderate', 'high']
# )
test_data['Fare_binning'] = pd.qcut(
    train_data.Fare,
    q=[0, 0.33, 0.66, 1],  # Quantiles
    labels=['low', 'moderate', 'high']
)

train_data['Age_binning'] = pd.cut(
    train_data.Fare,
    bins=[0, train_data.Age.quantile(0.33), train_data.Age.quantile(0.66), train_data.Fare.max()],
    labels=['child', 'young', 'old']
)
test_data['Age_binning'] = pd.cut(
    train_data.Fare,
    bins=[0, train_data.Age.quantile(0.33), train_data.Age.quantile(0.66), train_data.Fare.max()],
    labels=['child', 'young', 'old']
)





train_data


Name column

In [ ]:
train_data['Title'] = train_data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].replace(' ', ''))
test_data['Title'] = test_data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].replace(' ', ''))
print(train_data['Title'].nunique())
print(train_data['Title'].unique())

train_data.Title.replace(['Mrs' , 'Mlle' , 'Lady'],'Ms' , inplace=True)
test_data.Title.replace(['Mrs' , 'Mlle' , 'Lady'],'Ms' , inplace=True)

train_data.Title.replace(['Sir' , 'Mme'],'Mr' , inplace=True)
test_data.Title.replace(['Sir' , 'Mme'],'Mr' , inplace=True)

train_data['Title'] = train_data.Title.transform(lambda x: 'other' if x not in ['Ms' , 'Mr'] else x)

train_data['Title'].value_counts()

### Encoding categorical data

solution 1

In [ ]:
#map method
train_data['Sex']=train_data['Sex'].map({'female':0 , 'male':1} )
test_data['Sex']=test_data['Sex'].map({'female':0 , 'male':1})
train_data.sample(3)

solution 2

In [ ]:

# Columns to encode
columns_to_encode = ['Cabin', 'Ticket_Prefix', 'Title', 'Embarked', 'Ticket' ,'Is_Alone','Fare_binning' , 'Age_binning']

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse=False, drop='first', handle_unknown='infrequent_if_exist')

# Loop through each column and apply encoding
for column in columns_to_encode:
    # Add an "Unknown" category to the training data
    train_data[column] = train_data[column].astype(str)  # Ensure all values are strings
    train_data[column].fillna('Unknown', inplace=True)  # Replace NaN with "Unknown"

    # Fit and transform the training data
    train_encoded = encoder.fit_transform(train_data[[column]])
    train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out([column]))

    # Add an "Unknown" category to the test data
    test_data[column] = test_data[column].astype(str)  # Ensure all values are strings
    test_data[column].fillna('Unknown', inplace=True)  # Replace NaN with "Unknown"

    # Transform the test data
    test_encoded = encoder.transform(test_data[[column]])
    test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out([column]))

    # Concatenate the encoded data with the original DataFrame
    train_data = pd.concat([train_data, train_encoded_df], axis=1).drop(column, axis=1)
    test_data = pd.concat([test_data, test_encoded_df], axis=1).drop(column, axis=1)

# Display the transformed data
print("Transformed Train Data:")
print(train_data.head())

print("\nTransformed Test Data:")
print(test_data.head())

Train test split

In [ ]:
X = train_data.drop('Survived' , axis=1)
y = train_data['Survived']


train_data.columns

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2 , random_state=42)


In [ ]:
ssc = StandardScaler()
ssc.fit_transform(X_train.drop('Name' , axis=1))
ssc.transform(X_test.drop('Name' , axis=1))

outlier, skewness handling

In [ ]:
X[['Parch','Fare' , 'SibSp' , 'Age' , 'Pclass']].hist(bins=50, figsize=(50, 50), layout=(5, 2), grid=False)
plt.show()
num_tr_data.skew()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import boxcox

# Function to handle outliers
def outlier_handler(df):
    for feature in df.columns:
        Q1 = df[feature].quantile(0.25)  # First Quartile
        Q3 = df[feature].quantile(0.75)  # 3rd Quartile
        IQR = Q3 - Q1
        upper_bound = Q3 + 1.5 * IQR  # Upper bound
        lower_bound = Q1 - 1.5 * IQR  # Lower bound
        # Clip outliers
        df[feature] = df[feature].clip(lower_bound, upper_bound)
    return df

# Visualize the distribution of numerical features
X_train[['Parch', 'Fare', 'SibSp', 'Age', 'Pclass']].hist(bins=50, figsize=(20, 20), layout=(5, 2), grid=False)
plt.show()

# Check skewness
print("Skewness before handling:")
print(X_train[['Parch', 'Fare', 'SibSp', 'Age', 'Pclass']].skew())

# Handle skewness using log transformation (for positive values only)
def handle_skewness(df):
    for feature in df.columns:
        if df[feature].min() > 0:  # Log transformation works only for positive values
            df[feature] = np.log1p(df[feature])  # log1p to handle zeros
        else:
            # Use Box-Cox transformation for features with non-positive values
            df[feature], _ = boxcox(df[feature] + 1)  # Add 1 to handle zeros
    return df

# Apply skewness handling to X_train and X_test
X_train[['Parch', 'Fare', 'SibSp', 'Age', 'Pclass']] = handle_skewness(X_train[['Parch', 'Fare', 'SibSp', 'Age', 'Pclass']])
X_test[['Parch', 'Fare', 'SibSp', 'Age', 'Pclass']] = handle_skewness(X_test[['Parch', 'Fare', 'SibSp', 'Age', 'Pclass']])

# Check skewness after handling
print("\nSkewness after handling:")
print(X_train[['Parch', 'Fare', 'SibSp', 'Age', 'Pclass']].skew())

# Handle outliers using the outlier_handler function
X_train[['Parch', 'Fare', 'SibSp', 'Age', 'Pclass']] = outlier_handler(X_train[['Parch', 'Fare', 'SibSp', 'Age', 'Pclass']])
X_test[['Parch', 'Fare', 'SibSp', 'Age', 'Pclass']] = outlier_handler(X_test[['Parch', 'Fare', 'SibSp', 'Age', 'Pclass']])

# Visualize the distribution after handling skewness and outliers
X_train[['Parch', 'Fare', 'SibSp', 'Age', 'Pclass']].hist(bins=50, figsize=(20, 20), layout=(5, 2), grid=False)
plt.show()

### XGBoost

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 1: Check column types
print("X_train dtypes before conversion:")
print(X_train.dtypes)

print("\nX_test dtypes before conversion:")
print(X_test.dtypes)

# Step 2: Convert object columns to category or numeric
for col in X_train.columns:
    if X_train[col].dtype == 'object':  # If column is of type object
        X_train[col] = X_train[col].astype('category')  # Convert to category
        X_test[col] = X_test[col].astype('category')  # Convert to category

# Step 3: Verify column types after conversion
print("\nX_train dtypes after conversion:")
print(X_train.dtypes)

print("\nX_test dtypes after conversion:")
print(X_test.dtypes)

# Step 4: Initialize XGBoost Classifier with enable_categorical=True
xgb_model = xgb.XGBClassifier(
    n_estimators=500,  # Number of boosting rounds
    max_depth=5,       # Maximum depth of a tree
    learning_rate=0.1, # Learning rate
    objective='binary:logistic',  # For binary classification
    random_state=42,
    enable_categorical=True  # Enable support for categorical features
)

# Step 5: Train the model
xgb_model.fit(X_train, y_train)

# Step 6: Make predictions
y_train_pred = xgb_model.predict(X_train)
y_test_pred = xgb_model.predict(X_test)

# Step 7: Evaluate the model
# Training set performance
print("Training Accuracy:", accuracy_score(y_train, y_train_pred))
print("Training Classification Report:\n", classification_report(y_train, y_train_pred))
print("Training Confusion Matrix:\n", confusion_matrix(y_train, y_train_pred))

# Test set performance
print("\nTest Accuracy:", accuracy_score(y_test, y_test_pred))
print("Test Classification Report:\n", classification_report(y_test, y_test_pred))
print("Test Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))

# Step 8: Feature Importance
feature_importance = xgb_model.feature_importances_
feature_names = X_train.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("\nFeature Importance:")
print(importance_df)

# Step 9: Visualize Feature Importance
importance_df.plot(kind='bar', x='Feature', y='Importance', figsize=(10, 6), legend=False)
plt.title('Feature Importance')
plt.ylabel('Importance Score')
plt.show()